In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DateType
from pyspark.sql import functions as F
from pyspark.sql.functions import mean
import datetime
from pyspark.pandas import range
spark = (SparkSession.builder
        .appName("MySparkApp")
        .enableHiveSupport()
        .getOrCreate())

In [2]:
csvSchema = StructType([
    StructField("fecha", StringType(), True),
    StructField("horaUTC", StringType(), False),
    StructField("clase_de_vuelo", StringType(), True),
    StructField("clasificacion_de_vuelo", StringType(), True),
    StructField("tipo_de_movimiento", StringType(), True),
    StructField("aeropuerto", StringType(), True),
    StructField("origen_destino", StringType(), True),
    StructField("aerolinea_nombre", StringType(), True),
    StructField("aeronave", StringType(), True),
    StructField("pasajeros", IntegerType(), True)
])

# Vuelos

In [3]:
vuelos_2021 = (spark.read.option('header', 'true')
                         .option('delimiter', ';')
                         .csv('hdfs://172.17.0.2:9000/ingest/vuelos/2021-informe-ministerio.csv', schema=csvSchema))
vuelos_2022 = (spark.read.option('header', 'true')
                         .option('delimiter', ';')
                         .csv('hdfs://172.17.0.2:9000/ingest/vuelos/202206-informe-ministerio.csv', schema=csvSchema))
aeropuertos_detalle = (spark.read.option('header', 'true')
                            .option('delimiter', ';')
                            .csv('hdfs://172.17.0.2:9000/ingest/vuelos/aeropuertos_detalle.csv'))

vuelos_2021 = vuelos_2021.withColumn("fecha_as_date", F.to_date('fecha', 'dd/mm/yyyy'))
vuelos_2022 = vuelos_2022.withColumn("fecha_as_date", F.to_date('fecha', 'dd/mm/yyyy'))
# Unimos las dos tablas
vuelos = vuelos_2021.unionByName(vuelos_2022)


vuelos = vuelos.drop('fecha')
vuelos = vuelos.withColumnRenamed('fecha_as_date', 'fecha')
vuelos = vuelos.withColumn("pasajeros", col("pasajeros").cast
# Unimos las dos tablas
vuelos = vuelos_2021.union(vuelos_2022)


In [4]:
vuelos.select('clase_de_vuelo', 'clasificacion_de_vuelo', 'origen_destino', 'aeronave', 'pasajeros').show(10)

[Stage 1:>                                                          (0 + 1) / 1]

+--------------------+----------------------+--------------+----------------+---------+
|      clase_de_vuelo|clasificacion_de_vuelo|origen_destino|        aeronave|pasajeros|
+--------------------+----------------------+--------------+----------------+---------+
|Vuelo Privado con...|             Domestico|           ROS|    PA-PA-28-181|        0|
|             Regular|             Domestico|           GRA|     BO-B737-8MB|       70|
|             Regular|             Domestico|           ECA|      BO-737-800|       70|
|             Regular|             Domestico|           SAL|    BO-B-737-76N|       12|
|             Regular|             Domestico|           TUC|EMB-ERJ190100IGW|       26|
|             Regular|         Internacional|          SBGR|               0|       62|
|Vuelo Privado con...|             Domestico|           PAR|    PA-PA-28-181|        0|
|             Regular|         Internacional|          MMMX|               0|      198|
|             Regular|         I

# Numero de pasajeros en viajes domésticos por aeropuerto

In [5]:
vuelos_pasajeros_aeropuerto = (vuelos.filter(vuelos.clasificacion_de_vuelo == 'Domestico')
                               .groupby('aeropuerto')
                               .sum('pasajeros')
                               .orderBy('sum(pasajeros)', ascending=False))
vuelos_pasajeros_aeropuerto = vuelos_pasajeros_aeropuerto.withColumnRenamed('sum(pasajeros)', 'pasajeros')

vuelos_pasajeros_aeropuerto.show()

[Stage 2:=============================>                             (2 + 2) / 4]

+----------+---------+
|aeropuerto|pasajeros|
+----------+---------+
|       AER|  2155614|
|       EZE|   672938|
|       BAR|   558863|
|       CBA|   346664|
|       DOZ|   321230|
|       SAL|   258869|
|       USU|   255580|
|       IGU|   206224|
|       NEU|   194030|
|       ECA|   165105|
|       TUC|   160823|
|       CRV|    92205|
|       JUJ|    87822|
|       TRE|    84256|
|       POS|    72001|
|       CHP|    57691|
|       ROS|    51874|
|       CRR|    48426|
|       GAL|    45722|
|       MDP|    43968|
+----------+---------+
only showing top 20 rows



In [55]:
vuelos_pasajeros_aeropuerto.write.mode('overwrite').saveAsTable('aeropuertos.aeropuerto_tabla')

2025-06-11 10:07:40,358 WARN conf.HiveConf: HiveConf of name hive.metastore.local does not exist
2025-06-11 10:07:42,831 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


# Aeropuertos detalle

In [36]:
aeropuertos_detalle.columns

['local',
 'oaci',
 'iata',
 'tipo',
 'denominacion',
 'coordenadas',
 'latitud',
 'longitud',
 'elev',
 'uom_elev',
 'ref',
 'distancia_ref',
 'direccion_ref',
 'condicion',
 'control',
 'region',
 'fir',
 'uso',
 'trafico',
 'sna',
 'concesionado',
 'provincia',
 'inhab']

In [49]:
aeropuertos_detalle.select('condicion').show()

+---------+
|condicion|
+---------+
|  PRIVADO|
|  PUBLICO|
|  PRIVADO|
|  PUBLICO|
|  PRIVADO|
|  PRIVADO|
|  PUBLICO|
|  PRIVADO|
|  PUBLICO|
|  PRIVADO|
|  PRIVADO|
|  PRIVADO|
|  PRIVADO|
|  PRIVADO|
|  PUBLICO|
|  PRIVADO|
|  PRIVADO|
|  PRIVADO|
|  PRIVADO|
|  PUBLICO|
+---------+
only showing top 20 rows



In [6]:
# Cantidad de aeropuertos públicos por provincia
aeropuertos_publicos_provincia = (aeropuertos_detalle.filter(aeropuertos_detalle.condicion == 'PUBLICO')
                                  .groupBy('provincia')
                                  .count()
                                  .orderBy('count', ascending=False))
aeropuertos_publicos_provincia.show(10)                                 

+------------+-----+
|   provincia|count|
+------------+-----+
|BUENOS AIRES|  106|
|     CÓRDOBA|   33|
|    SANTA FÉ|   25|
|   RÍO NEGRO|   15|
|  ENTRE RÍOS|   14|
|      CHUBUT|   11|
|    LA PAMPA|   11|
|  CORRIENTES|    9|
|  SANTA CRUZ|    9|
|     NEUQUÉN|    8|
+------------+-----+
only showing top 10 rows



In [60]:
aeropuertos_publicos_provincia.write.mode('overwrite').saveAsTable('aeropuertos.aeropuerto_detalles_tabla')